# In this part, we will implement the BB84 protocol between Alice and Bob.

# Problem #2.1
Let's start by creating a BB84 object and ensuring the following attributes have been created:

* qubit
* simulator

In [9]:
# @title
import warnings
warnings.filterwarnings('ignore')


try:
    import cirq
except ImportError:
    print('installing cirq...')
    !pip install cirq --quiet
    import cirq
    print('installed cirq.')


#!git clone https://github.com/the-codingschool/bb84.git
#from bb84.bb84 import BB84
from cirq import NamedQubit, Simulator, Circuit

class BB84:

    """
    Implements the BB84 protocol for quantum key distribution.

    The BB84 protocol allows two parties, Alice and Bob, to generate a shared secret key, which can then be used
    for secure communication. The protocol also involves checking for the presence of an eavesdropper, Eve.
    This implementation provides a simulated environment for the protocol's execution, including the potential
    interception by Eve.


    <b>NOTE</b>:
        This class requires external definitions for the quantum circuits and simulator to function properly.

    <br>

    © 2024 The Coding School, All rights reserved.
    """

    qubit: NamedQubit
    """The qubit being used for the protocol. Must be defined externally, for example:
    ```python
    protocol.qubit = cirq.NamedQubit('q0')
    ```
    """

    simulator: Simulator
    """The quantum circuit simulator. Must be defined externally, for example:
    ```python
    protocol.simulator = cirq.Simulator()
    ```
    """

    alice_send_0_no_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 0 without an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_0_no_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_1_no_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 1 without an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_1_no_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_0_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 0 with an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_0_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_1_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 1 with an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_1_H_circuit = cirq.Circuit()
    ```
    """

    eve_intercept_circuit: Circuit
    """The quantum circuit that Eve will use to intercept the qubit between Phase 1 and 2. Must be defined externally, for example:
    ```python
    protocol.eve_intercept_circuit = cirq.Circuit()
    ```
    """

    bob_receive_no_H_circuit: Circuit
    """The quantum circuit that Bob will use to receive and measure the qubit during Phase 3 without using an H gate. Must be defined externally, for example:
    ```python
    protocol.bob_receive_no_H_circuit = cirq.Circuit()
    ```
    """

    bob_receive_H_circuit: Circuit
    """The quantum circuit that Bob will use to receive and measure the qubit during Phase 3 using an H gate. Must be defined externally, for example:
    ```python
    protocol.bob_receive_H_circuit = cirq.Circuit()
    ```
    """


    eve_intercept: str
    """Indicates if Eve intercepts the qubits ('yes' or 'no'). Should be set upon initialization or after calling `reset()`."""

    alice_bit: int
    """The current bit Alice wants to send. Should only be modified indirectly through the `send_bit` method's parameters."""

    bob_bit: int
    """The bit received by Bob after measurement. Should only be modified indirectly through the `send_bit` method's parameters."""

    eve_bit: int
    """The bit measured by Eve if interception occurs. Should only be modified indirectly through the `send_bit` method's parameters."""

    does_alice_apply_H: str
    """Indicates if Alice applies the Hadamard gate ('yes' or 'no'). Should only be modified indirectly through the `send_bit` method's parameters."""

    does_bob_apply_H: str
    """Indicates if Bob applies the Hadamard gate ('yes' or 'no'). Should only be modified indirectly through the `send_bit` method's parameters."""


    alice_key: list
    """The secret key generated by Alice. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    bob_key: list
    """The secret key generated by Bob. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    eve_key: list
    """The key intercepted by Eve, if any. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    bit_num: int
    """Counter for the number of bits processed so far. <b>SHOULD NOT BE MODIFIED BY USER</b>."""


    def __init__(self, eve_intercept = 'no'):

      """
      Initializes the BB84 protocol simulation with default or specified settings.

      Parameters:
          `eve_intercept` (str, optional): Determines if Eve will attempt to intercept the qubits ('yes' or 'no').
                                         Defaults to 'no'.

      Example Usage:
      ```python
      protocol_without_eve = BB84()

      also_protocol_without_eve = BB84(eve_intercept = 'no')

      protocol_with_eve = BB84(eve_intercept = 'yes')
      ```
      """

      self.alice_bit = None
      self.bob_bit = None

      self.eve_bit = None
      self.eve_intercept = eve_intercept

      self.does_alice_apply_H = None
      self.does_bob_apply_H = None

      self.qubit = None
      self.simulator = None

      self.alice_send_0_no_H_circuit = None
      self.alice_send_1_no_H_circuit = None
      self.alice_send_0_H_circuit = None
      self.alice_send_1_H_circuit = None

      self.eve_intercept_circuit = None

      self.bob_receive_no_H_circuit = None
      self.bob_receive_H_circuit = None

      self.alice_key = []
      self.bob_key = []
      self.eve_key = []

      self.bit_num = 1


    def phase_1_circuit(self):

      """
      Creates a circuit such that Alice encodes her bit into a qubit and applies an H gate if she's chosen to do so.
      Then she sends this qubit through her quantum channel to Bob, hoping Eve does not intercept.

      <b>Returns</b>:
          The quantum circuit for Phase 1 of BB84 (through Alice sending her qubit).

      <b>INTERNAL USE ONLY</b>
      """

      if self.alice_bit == 0:
        if self.does_alice_apply_H == 'no':
          return self.alice_send_0_no_H_circuit
        else:
          return self.alice_send_0_H_circuit

      else:
        if self.does_alice_apply_H == 'no':
          return self.alice_send_1_no_H_circuit
        else:
          return self.alice_send_1_H_circuit


    def eve_circuit(self):

      """
      Creates a circuit for Eve's role in the protocol, which depends on if she's intercepting or not.

      <b>Returns</b>:
          The quantum circuit for Eve's interception if it occurs, otherwise an empty circuit.

      <b>INTERNAL USE ONLY</b>
      """

      if self.eve_intercept == 'yes':
        return self.eve_intercept_circuit
      else:
        return cirq.Circuit()


    def phase_2_circuit(self):

      """
      Creates a circuit such that Bob receives the qubit, applies an H gate if he's decided to do so,
      and then measures it.

      <b>Returns</b>:
          The quantum circuit for Phase 2 of BB84 (through Bob measuring the qubit).

      <b>INTERNAL USE ONLY</b>
      """

      if self.does_bob_apply_H == 'no':
        return self.bob_receive_no_H_circuit
      else:
        return self.bob_receive_H_circuit


    def restart(self):

      """
      Resets the protocol such that the circuits are all the same, but they are being used with blank keys and on a new quantum channel.
      """

      self.alice_key = []
      self.bob_key = []
      self.eve_key = []

      self.bit_num = 1


    def send_bit(self, alice_bit, does_alice_apply_H, does_bob_apply_H, compare_bits = 'no'):

      """
      Simulates the full BB84 protocol, potentially intercepted by Eve.

      Parameters:
        <ul>
          <li>`alice_bit` (int): The bit Alice wants to send.</li>
          <li>`does_alice_apply_H` (str): Indicates if Alice applies the Hadamard gate ('yes' or 'no').</li>
          <li>`does_bob_apply_H` (str): Indicates if Bob applies the Hadamard gate ('yes' or 'no').</li>
          <li>`compare_bit` (str, optional): Determines if Alice and Bob compare their bits directly ('yes' or 'no').
                                       Defaults to 'no'.</li>
        </ul>

      <b>NOTE</b>:
          This method simulates the entire process of sending a bit, including preparation, potential interception,
          and measurement. It also handles error checking for uninitialized objects that are required in the given case.


      Example Usage:
      ```python
      # Alice sends a 1 without an H gate and Bob receives and measures the qubit without an H gate.
      # The default is that they will not compare their bits and instead will add them to their key
      # if they both made the same choice of H gate or not.
      protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'no', does_bob_apply_H = 'no')


      # Same as above, except Alice and Bob will compare bits instead of adding them to their key.
      protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'yes')
      ```
      """

      self.alice_bit = alice_bit
      self.does_alice_apply_H = does_alice_apply_H
      self.does_bob_apply_H = does_bob_apply_H

      if self.qubit == None:
        print('Error: A qubit object must be defined first.')
        return

      elif self.simulator == None:
        print('Error: A simulator object must be defined first.')
        return


      # Prepare Alice's qubit
      alice_circuit = self.phase_1_circuit()

      if alice_circuit == None:

        applies_H = 'no H'
        if self.does_alice_apply_H == 'yes':
          applies_H = 'an H'

        print('Error: Alice\'s circuit for sending a', self.alice_bit, 'with', applies_H, 'must be defined first.')
        return


      # Eve's interception
      eve_circuit = self.eve_circuit()

      if eve_circuit == None:

        print('Error: Eve\'s interception circuit must be defined first.')
        return

      # Bob's measurement
      bob_circuit = self.phase_2_circuit()

      if bob_circuit == None:

        applies_H = 'no H'
        if self.does_bob_apply_H == 'yes':
          applies_H = 'an H'

        print('Error: Bob\'s circuit for measuring with', applies_H, 'must be defined first.')
        return

      # Combine circuits and run simulation
      bb84_circuit = alice_circuit + eve_circuit + bob_circuit
      results = self.simulator.run(bb84_circuit)
      self.bob_bit = results.measurements['q0'][0][0]

      if self.eve_intercept:
        self.eve_bit = results.measurements.get('eve', [[None]])[0][0]

      # Print results
      print('\033[43m\033[1mATTEMPTED BIT', self.bit_num, '\033[0m\033[0m')

      print('\n\033[32m\033[1mPHASE 1: SENDING\033[0m\033[0m\033[0m')
      print('\033[47m\033[1mAlice (to herself)\033[0m\033[0m: I sent a', self.alice_bit, 'and', 'did not use' if self.does_alice_apply_H == 'no' else 'used', 'an H')

      if self.eve_intercept == 'yes':
        print('EVE INTERCEPTS!')
        print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: I measured a', self.eve_bit, 'and will now send the qubit to Bob')

      print('\n\033[32m\033[1mPHASE 2: RECEIVING\033[0m\033[0m')
      print('\033[47m\033[1mBob (to himself)\033[0m\033[0m: I', 'did not use' if self.does_bob_apply_H == 'no' else 'used', 'an H and measured a', self.bob_bit)

      print('\n\033[32m\033[1mPHASE 3: COMPARING\033[0m\033[0m')
      print('Alice and Bob are comparing choice of H\'s', 'and the bits themselves.' if compare_bits == 'yes' else 'but not the bits themselves.', '\n')
      print('\033[47m\033[1mAlice\033[0m\033[0m: I', 'did not use' if self.does_alice_apply_H == 'no' else 'used', 'an H')
      print('\033[47m\033[1mBob\033[0m\033[0m: I', 'did not use' if self.does_bob_apply_H == 'no' else 'used', 'an H')

      print('')
      if compare_bits == 'yes':
        print('\033[47m\033[1mAlice\033[0m\033[0m: I sent a', self.alice_bit)
        print('\033[47m\033[1mBob\033[0m\033[0m: I measured a', self.bob_bit, '\n')

        if self.does_alice_apply_H == self.does_bob_apply_H:
          if self.alice_bit == self.bob_bit:
            print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: Our bits match, so it doesn\'t seem like Eve is intercepting.')

            if self.eve_intercept == 'yes':
              print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: Mwuhaha, I\'ve gone undetected.')

          else:
            print('\033[91m\033[1mAlice and Bob: Our bits are different, so Eve must have intercepted! Let\'s start over with new keys and a new quantum channel!\033[0m\033[0m')
            self.restart()

        else:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: We made different choices, we should not use this bit.')

      else:
        if self.does_alice_apply_H == self.does_bob_apply_H:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: Great, let\'s add this bit to our keys.')
          self.alice_key += [self.alice_bit]
          self.bob_key += [self.bob_bit]

          if self.eve_intercept:
            self.eve_key += [self.eve_bit]
        else:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: We made different choices, we should not use this bit.')

      print('')
      print('\033[47m\033[1mAlice (to herself)\033[0m\033[0m: My key is now', self.alice_key)
      print('\033[47m\033[1mBob (to himself)\033[0m\033[0m: My key is now', self.bob_key)

      if self.eve_intercept == 'yes':
        print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: My key is now:', self.eve_key)


      print('\nThe circuit used this round:', bb84_circuit)

      self.bit_num += 1
      print('='*75, end='\n\n')

import matplotlib.pyplot as plt


installing cirq...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.2 MB/s eta 0:00:0

In [10]:
my_protocol = BB84()

my_protocol.qubit = cirq.NamedQubit('q0')
my_protocol.simulator = cirq.Simulator()

# Problem #2.2
Now, let's create a circuit that represents Alice encoding a 0 into a qubit and not applying an H gate.


This should be stored in the my_protocol.alice_send_0_no_H_circuit attribute.

In [11]:
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.I(my_protocol.qubit))

my_protocol.alice_send_0_no_H_circuit = circuit

# Problem #2.3
Now, let's create a circuit that represents Bob not applying an H gate and then measuring.



This should be stored in the my_protocol.bob_receive_no_H_circuit attribute.

In [12]:
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.measure(my_protocol.qubit))

my_protocol.bob_receive_no_H_circuit = circuit

# Problem #2.4
Lastly, let's test our code out by using the my_protocol.send_bit(...) method. Make sure to provide the following parameters to use our code from above:

* alice_bit = 0
* does_alice_apply_H = 'no'
* does_bob_apply_H = 'no'

**NOTE: If everything has been coded correctly, you should see a "script" for this attempt to send a bit.**

In [13]:
my_protocol.send_bit(alice_bit=0, does_alice_apply_H='no', does_bob_apply_H='no')

ATTEMPTED BIT 1 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0]
Bob (to himself): My key is now [0]

The circuit used this round: q0: ───I───I───I───M───



# Reflection
* Did Alice and Bob end up using this bit in their key and is it the same value (0 or 1)?

* Will this always be the case provided Alice and Bob make the same choices. In other words, if you run the code again should you expect the same result every time?

# Solution
* Yes, Alice and Bob ended up using this bit as part of their key and they both had a 0.

* Yes, since Alice and Bob both decided not to use an H gate AND there's no Eve interception they will always get the same result.

# Problem #2.5

Confirm your reflection answers by sending another bit with the same criteria (0 and no H's) 3 more times.

In [14]:
my_protocol.send_bit(alice_bit=0, does_alice_apply_H='no', does_bob_apply_H='no')
my_protocol.send_bit(alice_bit=0, does_alice_apply_H='no', does_bob_apply_H='no')
my_protocol.send_bit(alice_bit=0, does_alice_apply_H='no', does_bob_apply_H='no')

ATTEMPTED BIT 2 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0, 0]
Bob (to himself): My key is now [0, 0]

The circuit used this round: q0: ───I───I───I───M───

ATTEMPTED BIT 3 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0, 0, 0]
Bob (to himself): My key is now [0, 0, 0]

The circuit used this round: q0: ───I───I───I───M───

ATTE

# Problem #2.6
* At this point, Alice and Bob should both have keys that are 4 bits long that they believe are the same. However, they have not compared any bits to check for an eavesdropper yet.


* In the cell below, send another bit with the same criteria but also specifying that the parameter compare_bit = 'yes'.

In [17]:
my_protocol.send_bit(alice_bit=0, does_alice_apply_H='no', does_bob_apply_H='no')


ATTEMPTED BIT 5 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0, 0, 0, 0, 0]
Bob (to himself): My key is now [0, 0, 0, 0, 0]

The circuit used this round: q0: ───I───I───I───M───



# Reflection
* Did Alice and Bob detect an eavesdropper here? Should they have?

* Did Alice and Bob add this bit to their keys? Should they have?

# Solution
* No, they didn't detect an eavesdropper because their bits and their choices of using H or not were the same so everything is as it should be. In reality, there is no eavesdropper so there is no reason for the result to have been different.

* No, because they have publicly shared this bit, they will not use it as part of their keys. Even if they didn't detect an eavesdropper on the quantum channel, there could be one listening into this classical channel so it would not be wise of Alice and Bob to keep this bit. They only compare bits for the purpose of checking for an eavesdropper.

# Problem #2.7
Now, try sending a bit such that:

* Alice sends a 1.
* Alice and Bob both do not use an H.
* They do not compare bits.

In [18]:
my_protocol.send_bit(alice_bit=1, does_alice_apply_H='no', does_bob_apply_H='no')

Error: Alice's circuit for sending a 1 with no H must be defined first.


# Problem #2.8
You should have run into an error. In the space below, fix this error and try sending the bit again.

# Problem #2.9
In the space below, define the remaining attributes:

* alice_send_0_H_circuit
* alice_send_1_H_circuit
* bob_receive_H_circuit

In [19]:
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.H(my_protocol.qubit))

my_protocol.alice_send_0_H_circuit = circuit

In [20]:
circuit = cirq.Circuit()

circuit.append(cirq.X(my_protocol.qubit))
circuit.append(cirq.H(my_protocol.qubit))

my_protocol.alice_send_1_H_circuit = circuit

In [21]:
circuit = cirq.Circuit()

circuit.append(cirq.H(my_protocol.qubit))
circuit.append(cirq.measure(my_protocol.qubit))

my_protocol.bob_receive_H_circuit = circuit

# Problem #2.10
Now, perform the following transmissions (in this order):

* Alice sends a 0 with an H gate and Bob does not use an H gate.
* Alice sends a 1 with an H gate and Bob does not use an H gate.
* Alice sends a 1 with an H gate and Bob does use an H gate.

**NOTE: To ensure either Alice or Bob uses an H gate, you can set the corresponding parameter to 'yes' instead of 'no'.**

In [23]:
my_protocol.send_bit(alice_bit=0, does_alice_apply_H='yes', does_bob_apply_H='no')

ATTEMPTED BIT 7 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and used an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 1

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I used an H
Bob: I did not use an H

Alice and Bob: We made different choices, we should not use this bit.

Alice (to herself): My key is now [0, 0, 0, 0, 0]
Bob (to himself): My key is now [0, 0, 0, 0, 0]

The circuit used this round: q0: ───I───H───I───M───



In [24]:
my_protocol.send_bit(alice_bit=1, does_alice_apply_H='yes', does_bob_apply_H='no')

ATTEMPTED BIT 8 

PHASE 1: SENDING
Alice (to herself): I sent a 1 and used an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I used an H
Bob: I did not use an H

Alice and Bob: We made different choices, we should not use this bit.

Alice (to herself): My key is now [0, 0, 0, 0, 0]
Bob (to himself): My key is now [0, 0, 0, 0, 0]

The circuit used this round: q0: ───X───H───I───M───



In [25]:
my_protocol.send_bit(alice_bit=1, does_alice_apply_H='yes', does_bob_apply_H='yes')

ATTEMPTED BIT 9 

PHASE 1: SENDING
Alice (to herself): I sent a 1 and used an H

PHASE 2: RECEIVING
Bob (to himself): I used an H and measured a 1

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I used an H
Bob: I used an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0, 0, 0, 0, 0, 1]
Bob (to himself): My key is now [0, 0, 0, 0, 0, 1]

The circuit used this round: q0: ───X───H───H───M───

